In [1]:
# Install required libraries
# !pip install yfinance pandas

"""
현재가 + 이동 평균선, 거래량 이동 평균선 + 365일, 180일, 90일, 30일 기간 내 최고점, 최저점 및 역사상 신고가, 최저가
1분봉 데이터 + 1분봉 이동 평균선

Date: 날짜 (YYYY-MM-DD 형식)
Close: 종가
Volume: 거래량
SMA_X: X일 단순 이동 평균선 (SMA)
VMA_X: X일 거래량 이동 평균선 (VMA)
"""

import yfinance as yf
import pandas as pd
import os
from datetime import datetime, timedelta
import numpy as np
from scipy.stats import linregress

project_dir = 'csv'
os.makedirs(project_dir, exist_ok=True)

def calculate_moving_average(data, window):
    """
    이동 평균을 계산하는 함수. 데이터가 부족한 경우 현재까지의 평균을 사용.

    Args:
        data (pd.Series): 주식 종가 데이터 시리즈.
        window (int): 이동 평균을 계산할 기간.

    Returns:
        pd.Series: 이동 평균 데이터 시리즈.
    """
    return data.rolling(window=window, min_periods=1).mean()

def calculate_trend_line(x, y):
    """
    여러 개의 전저점을 기반으로 선형 회귀 추세선을 계산하는 함수.

    Args:
        x (pd.Series): 날짜 인덱스 (정수형 변환 필요).
        y (pd.Series): 최저점 데이터.

    Returns:
        pd.Series: 계산된 추세선 데이터.
    """
    x_numeric = np.arange(len(x))  # 날짜 인덱스를 정수형으로 변환 (0, 1, 2, ...)
    slope, intercept, _, _, _ = linregress(x_numeric, y)  # 선형 회귀 수행 (기울기와 절편 계산)
    return slope * x_numeric + intercept  # y = mx + b 형태의 추세선 값 반환


def calculate_slope(data):
    """
    이동 평균선의 기울기(전일 대비 변화량)를 계산하는 함수.

    Args:
        data (pd.Series): 이동 평균 데이터 시리즈.

    Returns:
        pd.Series: 기울기 데이터 시리즈.
    """
    return data.diff() / 2 # (data[1] - data[0]) / 2

def get_stock_data(ticker, start_date, end_date, interval='1d'):
    """
    주어진 주식 코드와 기간에 해당하는 주식 데이터를 받아오는 함수.

    Args:
        ticker (str): 주식 코드.
        start_date (str): 데이터의 시작 날짜 (YYYY-MM-DD 형식).
        end_date (str): 데이터의 종료 날짜 (YYYY-MM-DD 형식).
        interval (str): 데이터 간격 (1d, 1m 등).

    Returns:
        pd.DataFrame: 주식 데이터 프레임.
    """
    data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    data = data[['Close', 'Volume']]  # 종가 및 거래량 데이터 사용
    data = data.reset_index()
    data.columns = ['Date', 'Close', 'Volume']

    # NaN 값을 앞쪽 값으로 채우기
    # data.ffill(inplace=True)
    # data.bfill(inplace=True)  # 앞쪽에 값이 없을 경우 뒤쪽 값으로 채우기

    # 추가 데이터 계산
    data['730D_High'] = data['Close'].rolling(window=730, min_periods=1).max()
    data['730D_Low'] = data['Close'].rolling(window=730, min_periods=1).min()
    data['365D_High'] = data['Close'].rolling(window=365, min_periods=1).max()
    data['365D_Low'] = data['Close'].rolling(window=365, min_periods=1).min()
    data['180D_High'] = data['Close'].rolling(window=180, min_periods=1).max()
    data['180D_Low'] = data['Close'].rolling(window=180, min_periods=1).min()
    data['90D_High'] = data['Close'].rolling(window=90, min_periods=1).max()
    data['90D_Low'] = data['Close'].rolling(window=90, min_periods=1).min()
    data['30D_High'] = data['Close'].rolling(window=30, min_periods=1).max()
    data['30D_Low'] = data['Close'].rolling(window=30, min_periods=1).min()
    data['AllTime_High'] = data['Close'].cummax()
    data['AllTime_Low'] = data['Close'].cummin()

    # 이동평균선 데이터 계산
    ma_columns = {}
    slope_columns = {}
    for ma in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60] + list(range(70, 710, 10)):
        ma_columns[f'SMA_{ma}'] = calculate_moving_average(data['Close'], ma)
        ma_columns[f'VMA_{ma}'] = calculate_moving_average(data['Volume'], ma)
        slope_columns[f'Slope_SMA_{ma}'] = calculate_slope(ma_columns[f'SMA_{ma}'])
        slope_columns[f'Slope_VMA_{ma}'] = calculate_slope(ma_columns[f'VMA_{ma}'])

    ma_df = pd.DataFrame(ma_columns)
    slope_df = pd.DataFrame(slope_columns)
    data = pd.concat([data, ma_df, slope_df], axis=1)

    # 전저점 기반 추세선 계산 및 현재가 접촉 여부 확인
    for period in ['730D_Low', '365D_Low', '180D_Low', '90D_Low', '30D_Low', 'AllTime_Low']:
        lows_y = data[period].dropna()
        lows_x = data['Date'].dropna().reset_index(drop=True).index[:len(lows_y)]  # 정수 인덱스 변환 및 길이 조정
        
        if len(lows_x) > 1:
            trend_values = calculate_trend_line(lows_x, lows_y)
            data.loc[data.index[:len(trend_values)], f'Trend_Support_{period}'] = trend_values
        else:
            data[f'Trend_Support_{period}'] = np.nan
        
        # 현재가가 해당 추세선에 닿았는지 여부 확인
        data[f'Touch_Trend_Support_{period}'] = data['Close'] <= data[f'Trend_Support_{period}']

    # 전고점 기반 추세선 계산 및 현재가 돌파 여부 확인
    for period in ['730D_High', '365D_High', '180D_High', '90D_High', '30D_High', 'AllTime_High']:
        highs_y = data[period].dropna()
        highs_x = data['Date'].dropna().reset_index(drop=True).index[:len(highs_y)]  # 정수 인덱스 변환 및 길이 조정
        
        if len(highs_x) > 1:
            trend_values = calculate_trend_line(highs_x, highs_y)
            data.loc[data.index[:len(trend_values)], f'Trend_Resistance_{period}'] = trend_values
        else:
            data[f'Trend_Resistance_{period}'] = np.nan
        
        # 현재가가 해당 저항선에 닿았는지 여부 확인
        data[f'Touch_Trend_Resistance_{period}'] = data['Close'] >= data[f'Trend_Resistance_{period}']

    return data

def save_data_to_csv(data, filename):
    """
    주어진 데이터를 CSV 파일로 저장하는 함수.

    Args:
        data (pd.DataFrame): 저장할 데이터 프레임.
        filename (str): 저장할 CSV 파일의 이름.
    """
    data.to_csv(filename, index=False)
    print(f'Data saved to {filename}')

def fetch_and_save_stock_data(tickers, start_date, end_date, project_dir, data_type='training'):
    """
    여러 주식 데이터를 가져와 CSV 파일로 저장
    - tickers: 종목 리스트 (예: ['AAPL', 'TSLA', 'GOOGL'])
    - start_date: 시작 날짜 ('YYYY-MM-DD')
    - end_date: 종료 날짜 ('YYYY-MM-DD')
    - project_dir: 데이터 저장 폴더
    - data_type: 'training', 'test'
    """
    for ticker in tickers:
        filename = os.path.join(project_dir, f"{ticker}_{data_type}_data.csv")
        print(f"📥 Downloading {ticker} {data_type} data...")

        data = get_stock_data(ticker, start_date, end_date, interval='1d')
        save_data_to_csv(data, filename)

# S&P 500
ticker = '^GSPC'
# 학습 데이터
start_date = '2004-01-01'
end_date = '2023-03-01'
filename = os.path.join(project_dir, 'sp500_training_data.csv')
data = get_stock_data(ticker, start_date, end_date, interval='1d')
save_data_to_csv(data, filename)

# 테스트 데이터
start_date = '2023-03-02'
end_date = '2025-02-12'
filename = os.path.join(project_dir, 'sp500_test_data.csv')
data = get_stock_data(ticker, start_date, end_date, interval='1d')
save_data_to_csv(data, filename)

# 📌 저장할 주식 리스트
tickers = ['AAPL', 'TSLA', 'GOOGL', 'MSFT', 'AMZN']

# ✅ 학습 데이터 (2004-01-01 ~ 2023-03-01)
fetch_and_save_stock_data(tickers, '2004-01-01', '2023-03-01', project_dir, data_type='training')

# ✅ 테스트 데이터 (2023-03-02 ~ 2025-02-12)
fetch_and_save_stock_data(tickers, '2023-03-02', '2025-02-12', project_dir, data_type='test')

# # ✅ 1분봉 데이터 예제 (최근 30일)
# fetch_and_save_stock_data(['AAPL'], '2025-01-01', '2025-02-12', project_dir, interval='1m')


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^GSPC']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Data saved to csv\sp500_training_data.csv


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^GSPC']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Data saved to csv\sp500_test_data.csv
📥 Downloading AAPL training data...


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Data saved to csv\AAPL_training_data.csv
📥 Downloading TSLA training data...


KeyboardInterrupt: 